In [1]:
!nvidia-smi

Wed May 12 20:29:15 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  On   | 00000000:00:05.0 Off |                  Off |
| N/A   43C    P0    45W / 250W |   1388MiB / 32480MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
import os, time
import mmcv
from mmcv import Config
from mmdet.datasets import (build_dataloader, build_dataset, replace_ImageToTensor) # 앞의 2개 --> mmdet/datasets/builder.py에 존재 / 마지막 1개 --> 
from mmdet.models import build_detector #mmdet/models/builder.py에 존재
from mmdet.apis import train_detector, single_gpu_test # mmdet/apis/train.py, test.py에 존재
from mmcv.runner import load_checkpoint
from mmcv.parallel import MMDataParallel
import pandas as pd
from pandas import DataFrame
from pycocotools.coco import COCO
import numpy as np
import json, wandb

## train part

In [6]:
classes = ("UNKNOWN", "General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
# config file 들고오기
#cfg = Config.fromfile('../configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py')
#파일 교체 시 여기 이름 바꾸기
config_dir_name = 'hrnet'
config_file_name = 'fcos_hrnetv2p_w40_gn-head_mstrain_640-800_4x4_2x_coco'
cfg = Config.fromfile('../configs/'+config_dir_name+'/'+config_file_name+'.py')

PREFIX = '/opt/ml/input/data/'

# dataset 바꾸기
cfg.data.train.classes = classes
cfg.data.train.img_prefix = PREFIX
cfg.data.train.ann_file = PREFIX + 'train.json'
cfg.data.train.pipeline[2]['img_scale'] = (512, 512)

cfg.data.val.classes = classes
cfg.data.val.img_prefix = PREFIX
cfg.data.val.ann_file = PREFIX + 'val.json'
cfg.data.val.pipeline[1]['img_scale'] = (512, 512)

cfg.data.test.classes = classes
cfg.data.test.img_prefix = PREFIX
cfg.data.test.ann_file = PREFIX + 'test.json'
cfg.data.test.pipeline[1]['img_scale'] = (512, 512)

cfg.data.samples_per_gpu = 16    #batch size

cfg.seed=42 #42로 고정
cfg.gpu_ids = [0]

cfg.model.bbox_head.num_classes = 11    #class 개수 설정

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)

#working directory 이름 설정
working_dir_name = '../work_dirs/' + config_file_name +'_'+str(cfg.runner.max_epochs)
cfg.work_dir = working_dir_name


cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)

#working directory 이름 설정
working_dir_name = '../work_dirs/' + config_file_name +'_'+str(cfg.runner.max_epochs)
cfg.work_dir = working_dir_name


## wandb setting

In [8]:
group_name = 'hrnet'; project_name = 'gjtak'; run_name = config_file_name
config_list = {
    'epoch' : cfg.runner.max_epochs,
    'batch_size' :  cfg.data.samples_per_gpu,
    'optimizer' : cfg.optimizer,
    'optimizer_config' : cfg.optimizer_config,
    'lr_config' : cfg.lr_config,
    'bbox_head_loss_cls' : cfg.model.bbox_head.loss_cls
}
cfg.log_config.hooks[1].init_kwargs['group']=group_name # group name(option)
cfg.log_config.hooks[1].init_kwargs['name'] = run_name # run name
cfg.log_config.hooks[1].init_kwargs['config'] = config_list # config

### config 저장

In [9]:
# config 객체 -> 파일(json)로 저장(저장 장소 = working directory와 같이)
#working directory 없으면 생성하기
try:
    if not os.path.exists(working_dir_name):
        os.makedirs(working_dir_name)
except OSError :
    print('Error: Creating directory :: ' + working_dir_name)

#cfg 객체 -> string(str이용) -> dict(eval이용) -> json 으로 저장
cfg_str = str(cfg)[str(cfg).find(')')+3:]
cfg_dict = eval(cfg_str)
with open(working_dir_name+'/config.json','w') as config_json_file :
    json.dump(cfg_dict, config_json_file, indent="\t")


In [11]:
#config 저장 확인
with open(working_dir_name+'/config.json', 'r') as f :
    json_data = json.load(f)
#json_data

In [12]:
model = build_detector(cfg.model)

2021-05-12 17:10:40,638 - mmdet - INFO - load model from: open-mmlab://msra/hrnetv2_w40
2021-05-12 17:10:40,639 - mmdet - INFO - Use load_from_openmmlab loader
2021-05-12 17:10:41,209 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: incre_modules.0.0.conv1.weight, incre_modules.0.0.bn1.weight, incre_modules.0.0.bn1.bias, incre_modules.0.0.bn1.running_mean, incre_modules.0.0.bn1.running_var, incre_modules.0.0.bn1.num_batches_tracked, incre_modules.0.0.conv2.weight, incre_modules.0.0.bn2.weight, incre_modules.0.0.bn2.bias, incre_modules.0.0.bn2.running_mean, incre_modules.0.0.bn2.running_var, incre_modules.0.0.bn2.num_batches_tracked, incre_modules.0.0.conv3.weight, incre_modules.0.0.bn3.weight, incre_modules.0.0.bn3.bias, incre_modules.0.0.bn3.running_mean, incre_modules.0.0.bn3.running_var, incre_modules.0.0.bn3.num_batches_tracked, incre_modules.0.0.downsample.0.weight, incre_modules.0.0.downsample.1.weight, incre_modules.0

In [13]:
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=3.91s)
creating index...
index created!


In [14]:
start = time.time()  # 시작 시간 저장
train_detector(model, datasets[0], cfg, distributed=False, validate=True)
print("time(sec) :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간(단위 : 초)

loading annotations into memory...
2021-05-12 17:10:56,987 - mmdet - INFO - Start running, host: root@7abc6b7bc976, work_dir: /opt/ml/p3-ims-obd-hansarang/gjtak_works/Object Detection/code/mmdetection_trash/work_dirs/fcos_hrnetv2p_w40_gn-head_mstrain_640-800_4x4_2x_coco_24
2021-05-12 17:10:56,988 - mmdet - INFO - workflow: [('train', 1)], max: 24 epochs
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
Done (t=0.90s)
creating index...
index created!
wandb: Currently logged in as: pstage3_det (use `wandb login --relogin` to force relogin)


18:32:11,195 - mmdet - INFO - Epoch [21][60/164]	lr: 1.000e-03, eta: 0:10:44, time: 1.090, data_time: 0.018, memory: 18190, loss_cls: 0.2114, loss_bbox: 0.2924, loss_centerness: 0.5902, loss: 1.0940, grad_norm: 2.1468
2021-05-12 18:32:22,090 - mmdet - INFO - Epoch [21][70/164]	lr: 1.000e-03, eta: 0:10:33, time: 1.090, data_time: 0.019, memory: 18190, loss_cls: 0.2031, loss_bbox: 0.3026, loss_centerness: 0.5911, loss: 1.0969, grad_norm: 2.0698
2021-05-12 18:32:32,993 - mmdet - INFO - Epoch [21][80/164]	lr: 1.000e-03, eta: 0:10:22, time: 1.090, data_time: 0.019, memory: 18190, loss_cls: 0.1957, loss_bbox: 0.3015, loss_centerness: 0.5904, loss: 1.0876, grad_norm: 1.8013
2021-05-12 18:32:43,903 - mmdet - INFO - Epoch [21][90/164]	lr: 1.000e-03, eta: 0:10:11, time: 1.091, data_time: 0.020, memory: 18190, loss_cls: 0.2111, loss_bbox: 0.2706, loss_centerness: 0.5878, loss: 1.0694, grad_norm: 2.1239
2021-05-12 18:32:54,840 - mmdet - INFO - Epoch [21][100/164]	lr: 1.000e-03, eta: 0:10:00, time:

train/loss_cls,0.20931
train/loss_bbox,0.26566
train/loss_centerness,0.58638
train/loss,1.06136
train/grad_norm,1.91296
learning_rate,0.0001
momentum,0.9
_runtime,5760
_timestamp,1620845217
_step,3937
val/bbox_mAP,0.112


train/loss_cls,▃▂▂▂▂▃█▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/loss_bbox,█▇▆▆▃▅▄▃▃▃▃▃▃▃▂▂▂▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
train/loss_centerness,██▇▆▅▆█▇▆▅▄▄▄▄▃▃▃▃▃▂▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
train/loss,█▆▆▆▃▅█▄▄▃▃▃▃▃▂▂▂▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
train/grad_norm,▆▂▂▃▃█▅▄▅▅▄▅▅▄▄▃▂▅▃▃▄▄▂▄▄▄▃▁▁▁▁▁▁▁▁▁▁▁▁▁
learning_rate,▃▃▃▃▃██████████████████████▂▂▂▂▂▂▂▂▂▁▁▁▁
momentum,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
val/bbox_mAP,▁▁▂▁▁▂▃▃▂▃▄▄▅▆▆▅████████


time(sec) : 5769.765910387039


## Inference part

In [15]:
epoch = cfg.runner.max_epochs
cfg.model.train_cfg = None

# checkpoint path
checkpoint_path = os.path.join(cfg.work_dir, f'epoch_{epoch}.pth')

In [16]:
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [17]:
model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg'))
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu')

model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

2021-05-12 18:47:02,376 - mmdet - INFO - load model from: open-mmlab://msra/hrnetv2_w40
2021-05-12 18:47:02,377 - mmdet - INFO - Use load_from_openmmlab loader
2021-05-12 18:47:02,927 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: incre_modules.0.0.conv1.weight, incre_modules.0.0.bn1.weight, incre_modules.0.0.bn1.bias, incre_modules.0.0.bn1.running_mean, incre_modules.0.0.bn1.running_var, incre_modules.0.0.bn1.num_batches_tracked, incre_modules.0.0.conv2.weight, incre_modules.0.0.bn2.weight, incre_modules.0.0.bn2.bias, incre_modules.0.0.bn2.running_mean, incre_modules.0.0.bn2.running_var, incre_modules.0.0.bn2.num_batches_tracked, incre_modules.0.0.conv3.weight, incre_modules.0.0.bn3.weight, incre_modules.0.0.bn3.bias, incre_modules.0.0.bn3.running_mean, incre_modules.0.0.bn3.running_var, incre_modules.0.0.bn3.num_batches_tracked, incre_modules.0.0.downsample.0.weight, incre_modules.0.0.downsample.1.weight, incre_modules.0

In [18]:
output = single_gpu_test(model, data_loader, show_score_thr=0.05)

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 837/837, 14.8 task/s, elapsed: 56s, ETA:     0s

In [19]:
prediction_strings = []
file_names = []
coco = COCO(cfg.data.test.ann_file)
imag_ids = coco.getImgIds()

class_num = 11
for i, out in enumerate(output):
    prediction_string = ''
    image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
    for j in range(class_num):
        for o in out[j]:
            prediction_string += str(j) + ' ' + str(o[4]) + ' ' + str(o[0]) + ' ' + str(o[1]) + ' ' + str(
                o[2]) + ' ' + str(o[3]) + ' '
        
    prediction_strings.append(prediction_string)
    file_names.append(image_info['file_name'])


submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names

submission.to_csv(os.path.join(cfg.work_dir, 'submission_'+config_file_name+f'_{epoch}.csv'), index=None)
submission.head()

loading annotations into memory...
Done (t=0.53s)
creating index...
index created!


,PredictionString,image_id
0,0 0.040290806 128.12572 54.976624 257.18927 17...,batch_01_vt/0021.jpg
1,0 0.0622886 2.6855164 244.7976 296.18066 458.5...,batch_01_vt/0028.jpg
2,0 0.08864036 65.54167 165.637 376.50534 386.05...,batch_01_vt/0031.jpg
3,0 0.043949112 287.4513 149.0187 359.8841 231.8...,batch_01_vt/0032.jpg
4,1 0.25261065 387.64752 273.28253 473.24817 346...,batch_01_vt/0070.jpg
